In [1]:
from src.runner import preprocess
preprocess()

/Users/Hanita/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing Center for Advanced Life Cycle Engineering cells: 100%|██████████| 7/7 [00:01<00:00,  4.11it/s]


0 processed, 7 skipped



Processing Hawaii Natural Energy Institute cells: 100%|██████████| 15/15 [00:03<00:00,  3.80it/s]


0 processed, 15 skipped



Processing Oxford cells: 100%|██████████| 8/8 [00:01<00:00,  5.32it/s]


...

230 processed, 23 skipped



Processing Sandia National Lab cells: 100%|██████████| 32/32 [00:04<00:00,  7.25it/s]


74 processed, 12 skipped



Processing Underwriters Lab - Purdue University cells: 100%|██████████| 22/22 [00:03<00:00,  7.03it/s]

21 processed, 1 skipped



In [7]:
from src.data.battery_data import BatteryData
oxford_data = BatteryData.load('data/preprocessed/oxford/OX_1-1_pouch_LCO_40C_0-100_2-1.84C_a_timeseries.pkl')
oxford_data.print_description()
ts = next(t for t in oxford_data.timeseries_data)
ts.display()

**************description of battery cell OX_1-1_pouch_LCO_40C_0-100_2-1.84C_a_timeseries**************
cell_id: OX_1-1_pouch_LCO_40C_0-100_2-1.84C_a_timeseries
organization: oxford
timeseries_data: [<src.preprocessing.battery_data.TimeseriesData object at 0x33426fdd0>]
is_healthy: True
anode_material: graphite
cathode_material: LCO
nominal_capacity_in_Ah: 0.72
form_factor: pouch


,Time (s),Temp (°C)
0,0.000000e+00,40.026
1,9.990000e-01,40.051
2,1.999000e+00,39.988
3,2.999000e+00,39.976
4,3.999000e+00,39.964
...,...,...
460376,5.518128e+07,40.737
460377,5.518128e+07,40.750
460378,5.518128e+07,40.688
460379,5.518128e+07,40.725


In [8]:
from src.data.battery_data import BatteryData
filename = 'data/preprocessed/oakridge/SNL_LFP_Graphite_10Ah_100SOC_b.pkl'
data = BatteryData.load(filename)
data.print_description()
ts = next(t for t in data.timeseries_data)
ts.display()

**************description of battery cell SNL_LFP_Graphite_10Ah_100SOC_b**************
cell_id: SNL_LFP_Graphite_10Ah_100SOC_b
organization: snl
timeseries_data: [<src.preprocessing.battery_data.TimeseriesData object at 0x1766e6120>, <src.preprocessing.battery_data.TimeseriesData object at 0x33427c200>, <src.preprocessing.battery_data.TimeseriesData object at 0x33427ca10>, <src.preprocessing.battery_data.TimeseriesData object at 0x33427fef0>, <src.preprocessing.battery_data.TimeseriesData object at 0x33427d880>, <src.preprocessing.battery_data.TimeseriesData object at 0x33427fd10>]
is_healthy: False
state_of_charge: 100
battery_type: (None,)
anode_material: graphite
cathode_material: LFP
nominal_capacity_in_Ah: 10.0
form_factor: cylindrical_18650


,Time (s),Temp (°C)
0,0.000000,22.364101
1,0.099609,22.358707
2,0.199219,22.365358
3,0.298828,22.381090
4,0.398438,22.315783
...,...,...
38712,3856.078125,29.106949
38713,3856.177734,29.090010
38714,3856.277344,29.108253
38715,3856.376953,29.104151


In [13]:
from pathlib import Path 
dir = Path(f'data/preprocessed/oakridge/')
cells = set(
    f.stem.split()[0]
    for f in dir.glob('*SNL_*')
    if f.is_file()
)

cells2 = set(
    f.stem.split()[0]
    for f in dir.glob('*')
    if f.is_file()
)

len(cells2), len(cells), len(cells2) - len(cells)

(230, 55, 175)

In [6]:
from sklearn.model_selection import train_test_split
from src.data.battery_data import BatteryData, TimeseriesData
from pathlib import Path
from src.config import config
from glob import glob 
import os


filters = [
    'org',
    'description contains punch'

    'soc, battery type, anode, cathode, ah, form factor'
]

def hkh_traintestsplit(preprocessed_dir = config.PROCESSED_DATA_DIR, orgs_to_include = None, timeseries_filter = None):
    ''
    # data = BatteryData.load(filename)
    parentdir = Path(preprocessed_dir)
    # for f in parentdir.glob('*SNL*'): print(f)

    features, labels = [], []
    for filename in glob(os.path.join(preprocessed_dir, '*SNL*')):
        filepath = os.path.join(preprocessed_dir, filename)
        cell = BatteryData.load(filepath)
        if orgs_to_include != None and cell.organization not in orgs_to_include:
            continue 

        X_temp, y_temp = extract_attributes_from_cell(cell, timeseries_filter=timeseries_filter)
        features.append(X_temp)
        labels.append(y_temp)
        print(filepath)

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

    



def extract_attributes_from_cell(cell: BatteryData, orgs_to_include = None, timeseries_filter = None):
    """Return a list of feature, label objects"""
    # if orgs_to_include != None and cell.organization not in orgs_to_include:
    #     return None, None 
    
    features = []
    for data in cell.timeseries_data:
        if timeseries_filter != None and any(term in data.description for term in timeseries_filter):
            continue 
        
        features.append(data.to_df())
        
    return features, [cell.is_healthy] * len(features)

filename = 'data/preprocessed/SNL_LFP_Graphite_10Ah_100SOC_b.pkl'    
cell = BatteryData.load(filename)
# extract_attributes_from_cell(cell, timeseries_filter=['punch'])
hkh_traintestsplit()


data/preprocessed/data/preprocessed/SNL_18650_NCA_25C_20-80_0.5-0.5C_a_timeseries.pkl
data/preprocessed/data/preprocessed/SNL_NMC-LMO_Graphite_26Ah_30SOC_b.pkl
data/preprocessed/data/preprocessed/SNL_LMO-LNO_Graphite_33Ah_50SOC_a.pkl
data/preprocessed/data/preprocessed/SNL_LFP_Graphite_10Ah_50SOC_a.pkl
data/preprocessed/data/preprocessed/SNL_LCO_Graphite_6.4Ah_70SOC_b.pkl
data/preprocessed/data/preprocessed/SNL_LFP_Graphite_10Ah_100SOC_b.pkl
data/preprocessed/data/preprocessed/SNL_18650_NCA_35C_0-100_0.5-1C_b_timeseries.pkl
data/preprocessed/data/preprocessed/SNL_NMC_Graphite_10Ah_70SOC_a.pkl
data/preprocessed/data/preprocessed/SNL_18650_NMC_25C_0-100_0.5-1C_b_timeseries.pkl
data/preprocessed/data/preprocessed/SNL_NMC-LMO_Graphite_26Ah_30SOC_a.pkl
data/preprocessed/data/preprocessed/SNL_18650_NMC_15C_0-100_0.5-2C_b_timeseries.pkl
data/preprocessed/data/preprocessed/SNL_LMO-LNO_Graphite_33Ah_50SOC_b.pkl
data/preprocessed/data/preprocessed/SNL_18650_NMC_25C_20-80_0.5-0.5C_a_timeseries.pk

In [14]:
[True] * 5

[True, True, True, True, True]